In [6]:
import json
import random
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.graph import StateGraph, MessagesState, START, END, MessageGraph
from IPython.display import display, Image
from typing import TypedDict, Annotated, Literal
from pydantic import BaseModel, Field
import operator
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode

load_dotenv("../.env")
llm = ChatGroq(
    # model="llama3-groq-70b-8192-tool-use-preview",
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)
#embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

memory = MemorySaver()
config = {"configurable": {"thread_id": 1}}

### Tool Calling

In [3]:
@tool
def get_current_weather(location:str)->str:
    """
    Get the current weather in a specific location.
    This tool simulates checking the weather by randomly selecting from three possible outcomes: Sunny, cold or rainy.
    The chance of each outcome is equal (1/3). If the random check fails, it may return an unexpected result to simulate.
    
    Args:
        location (str): The location for which the weather is to be checked.
        
    Returns:
        str: A string describing the weather in the given location, randomly chosen from three possible outcomes.
    """
    if random.randint(0, 2) == 0:
        return "Sunny, 78F"
    elif random.randint(0, 2) == 1:
        return "Cold, 22F"
    else:
        return "Rainy, 65F"
    

@tool
def get_system_time(location:str="Berlin")->str:
    """
    Get the system time. If no location is provided use the default location as "Minesota"
    This tool simulates retrieving the system time by randomly selecting from three possible outcomes: morning, afternoon, evenning.
    The chance of each outcome is equal (1/3). If the random check fails, it may return an unexpected result to simulate.
    
    Args:
        location (str, optional): The location for which the time is to be retrieved. Defaults to "Minesota".
        
    Returns:
        str: A string describing the time of day, randomly chosen from three possible outcomes.
    """
    
    if random.randint(0, 2) == 0:
        return "2:00 AM"
    elif random.randint(0, 2) == 1:
        return "3:00 PM"
    else:
        return "6:00 PM"

In [ ]:
tools = [get_current_weather, get_system_time]
llm_with_tools = llm.bind_tools(tools)

tool_mapping = {
    "get_current_weather": get_current_weather,
    "get_system_time": get_system_time
}

In [ ]:
# Define prompt
prompt = PromptTemplate(
    template="""system
    You are a smart Agent. 
    You are a master at understanding what a customer wants and utilize available tools only if you have to.

    user
    Conduct a comprehensive analysis of the request provided\

    USER REQUEST:\n\n {initial_request} \n\n
    
    assistant
    """,
    input_variables=["initial_request"],
)

agent_request_generator = prompt | llm_with_tools

result = agent_request_generator.invoke(
    {"initial_request": "What is the weather in woodbury in MN?"}
    )
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_qvqj', 'function': {'arguments': '{"location": "Woodbury, MN"}', 'name': 'get_current_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 737, 'total_tokens': 755, 'completion_time': 0.072, 'prompt_time': 0.155655706, 'queue_time': 0.21504934199999998, 'total_time': 0.227655706}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-d9e2422e-c751-4d59-8cd7-80c8377101c3-0' tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'Woodbury, MN'}, 'id': 'call_qvqj', 'type': 'tool_call'}] usage_metadata={'input_tokens': 737, 'output_tokens': 18, 'total_tokens': 755}


In [14]:

# Extract the last AI message from messages
last_ai_message = None
if isinstance(result, AIMessage):
    last_ai_message = result

# print("Last AI Message:", last_ai_message)

if last_ai_message and hasattr(last_ai_message, 'tool_calls'):
    print("last_ai_message:", last_ai_message)
    print("tool name :: ", last_ai_message.tool_calls[-1]["name"])
    print("tool args :: ", last_ai_message.tool_calls[-1]["args"])
else:
    print("No tool calls found.")

last_ai_message: content='' additional_kwargs={'tool_calls': [{'id': 'call_qvqj', 'function': {'arguments': '{"location": "Woodbury, MN"}', 'name': 'get_current_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 737, 'total_tokens': 755, 'completion_time': 0.072, 'prompt_time': 0.155655706, 'queue_time': 0.21504934199999998, 'total_time': 0.227655706}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-d9e2422e-c751-4d59-8cd7-80c8377101c3-0' tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'Woodbury, MN'}, 'id': 'call_qvqj', 'type': 'tool_call'}] usage_metadata={'input_tokens': 737, 'output_tokens': 18, 'total_tokens': 755}
tool name ::  get_current_weather
tool args ::  {'location': 'Woodbury, MN'}


In [16]:
result = agent_request_generator.invoke(
    {"initial_request": "What is the time in woodbury in MN?"}
    )
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_78yz', 'function': {'arguments': '{"location": "Woodbury, MN"}', 'name': 'get_system_time'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 737, 'total_tokens': 755, 'completion_time': 0.072, 'prompt_time': 0.136998677, 'queue_time': 0.003663271999999995, 'total_time': 0.208998677}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-aa08916b-7b61-491b-896b-678951680ed4-0' tool_calls=[{'name': 'get_system_time', 'args': {'location': 'Woodbury, MN'}, 'id': 'call_78yz', 'type': 'tool_call'}] usage_metadata={'input_tokens': 737, 'output_tokens': 18, 'total_tokens': 755}


In [17]:

# Extract the last AI message from messages
last_ai_message = None
if isinstance(result, AIMessage):
    last_ai_message = result

# print("Last AI Message:", last_ai_message)

if last_ai_message and hasattr(last_ai_message, 'tool_calls'):
    print("last_ai_message:", last_ai_message)
    print("tool name :: ", last_ai_message.tool_calls[-1]["name"])
    print("tool args :: ", last_ai_message.tool_calls[-1]["args"])
else:
    print("No tool calls found.")

last_ai_message: content='' additional_kwargs={'tool_calls': [{'id': 'call_78yz', 'function': {'arguments': '{"location": "Woodbury, MN"}', 'name': 'get_system_time'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 737, 'total_tokens': 755, 'completion_time': 0.072, 'prompt_time': 0.136998677, 'queue_time': 0.003663271999999995, 'total_time': 0.208998677}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-aa08916b-7b61-491b-896b-678951680ed4-0' tool_calls=[{'name': 'get_system_time', 'args': {'location': 'Woodbury, MN'}, 'id': 'call_78yz', 'type': 'tool_call'}] usage_metadata={'input_tokens': 737, 'output_tokens': 18, 'total_tokens': 755}
tool name ::  get_system_time
tool args ::  {'location': 'Woodbury, MN'}
